In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

4431872/4422102 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [ ]:
# 라벨 설정
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# 채널 값을 부여 : 28, 28 -> 28, 28, 1
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
train_images.shape, test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [ ]:
# AlexNet 모델 입력층에 맞게 채널 3장으로 맞춤
train_images = tf.repeat(train_images, 3, axis=3)
test_images = tf.repeat(test_images, 3, axis=3)
train_images.shape, test_images.shape

(TensorShape([60000, 28, 28, 3]), TensorShape([10000, 28, 28, 3]))

In [ ]:
model = models.Sequential()

# 227 x 227 x 3으로 resize
model.add(layers.experimental.preprocessing.Resizing(227, 227, 
                                                     input_shape=train_images.shape[1:]))

model.add(layers.Conv2D(96, (11, 11), strides=4, activation='relu')) # imagesize에 맞게 큰 filter에 큰 stride 적용
model.add(layers.Lambda(tf.nn.local_response_normalization)) # ReLU 결과값 안정화
model.add(layers.MaxPooling2D((3, 3), strides=2)) # Overlapping Pooling

model.add(layers.Conv2D(256, (5, 5), strides=1, activation='relu', padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))

model.add(layers.Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))

model.add(layers.Conv2D(256, (3, 3), strides=1, activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Flatten())
# model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

# model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(1000, activation='softmax'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_5 (Resizing)       (None, 227, 227, 1)       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 55, 55, 96)        11712     
                                                                 
 lambda_10 (Lambda)          (None, 55, 55, 96)        0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 lambda_11 (Lambda)          (None, 27, 27, 256)       0         
                                                      

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 65s 172ms/step - loss: 0.6119 - accuracy: 0.7692 - val_loss: 0.3519 - val_accuracy: 0.8693
Epoch 2/10
375/375 [==============================] - 64s 171ms/step - loss: 0.3393 - accuracy: 0.8754 - val_loss: 0.2968 - val_accuracy: 0.8898
Epoch 3/10
375/375 [==============================] - 64s 171ms/step - loss: 0.3040 - accuracy: 0.8868 - val_loss: 0.3028 - val_accuracy: 0.8838
Epoch 4/10
375/375 [==============================] - 64s 170ms/step - loss: 0.2665 - accuracy: 0.8997 - val_loss: 0.2628 - val_accuracy: 0.9049
Epoch 5/10
375/375 [==============================] - 64s 171ms/step - loss: 0.2451 - accuracy: 0.9086 - val_loss: 0.2703 - val_accuracy: 0.9013
Epoch 6/10
375/375 [==============================] - 64s 171ms/step - loss: 0.2312 - accuracy: 0.9135 - val_loss: 0.2781 - val_accuracy: 0.8977
Epoch 7/10
375/375 [==============================] - 64s 171ms/step - loss: 0.2225 - accuracy: 0.9165 - val_loss: 0.2621 - val_ac

In [ ]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 5s 15ms/step - loss: 0.2764 - accuracy: 0.9050


[0.2764274775981903, 0.9049999713897705]

In [ ]:
# AlexNet 모델을 간소화해봅시다 
## resize없이 (28, 28, 3)으로 받아봅시다

## ** activation 은 모두 relu로 유지
## layer 1 : 32개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

## layer 2 : 64개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

## layer 3 : 128개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x

## layer 4, 5 : 삭제

## fully connected layer 1, 2 : node 원하는대로 주기
